**Notebook was motivated in part by https://github.com/lsst/rubin_sim_notebooks/blob/main/maf/tutorial/Survey_footprint.ipynb**

**Import relevant python packages:**

In [ ]:
import os
import numpy as np
import pandas as pd
import sqlite3

# import rubin_sims maf packages 
import rubin_sim.maf as maf
from rubin_sim.data import get_baseline

import matplotlib.pyplot as plt

**Grab sqlite3 database file for the simulation of the baseline survey:**

In [ ]:
opsdb = get_baseline()
print(opsdb)

opsim = os.path.basename(opsdb).replace('.db','')
print(opsim)


**Old stuff on Metric Analysis Framework ("MAF"):**

In [ ]:
#metric = maf.metrics.CountMetric(col='observationStartMJD', metric_name='Nvisits')
#nside = 16
#slicer = maf.slicers.HealpixSlicer(nside=nside)
#constraint = None

**Get list of tables from the opsdb database:**

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(opsdb)

# Create a cursor object
cursor = conn.cursor()

# Execute a query to get the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all results
tables = cursor.fetchall()

# Print the list of tables
for table in tables:
    print(table[0])  # Each table name is in a tuple

# Close the connection
conn.close()

**Get the schema for the observations table from the opsdb database:**

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(opsdb)

# Create a cursor object
cursor = conn.cursor()

# Execute a query to get the schema for the "observations" table
cursor.execute("PRAGMA table_info(observations);")

# Fetch all results
schema = cursor.fetchall()

# Print the schema
for column in schema:
    print(f"Column Name: {column[1]}, Data Type: {column[2]}, Not Null: {column[3]}, Default Value: {column[4]}")

# Close the connection
conn.close()

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (replace 'example.db' with your database file)
conn = sqlite3.connect(get_baseline())

# Create a cursor object
cursor = conn.cursor()

# Execute a query and save results into a pandas dataframe...
df = pd.read_sql_query("SELECT observationId, fieldRA, fieldDec, observationStartMJD, filter FROM observations", conn)

# Close the connection
conn.close()

In [ ]:
df.head()

**Plot fieldRA, fieldDec for "g" band:**

In [ ]:
mask = (df['filter']=="g")
df[mask].plot('fieldRA','fieldDec', kind='scatter', style=".", s=0.1)

**Create methods for plotting an all-sky Aitoff projection map (from Claude-3.5-Sonnet):**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u

def create_sky_plot(df, ra_col='RA', dec_col='DEC', 
                   color_col=None, size_col=None, 
                   figsize=(12, 8), alpha=0.5,
                   title='All-Sky Aitoff Projection'):
    """
    Create an all-sky Aitoff projection plot from DataFrame coordinates.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing the coordinate data
    ra_col : str
        Name of the RA column (in degrees)
    dec_col : str
        Name of the DEC column (in degrees)
    color_col : str, optional
        Name of column to use for point colors
    size_col : str, optional
        Name of column to use for point sizes
    figsize : tuple
        Figure size in inches
    alpha : float
        Transparency of points
    title : str
        Plot title
    """
    
    # Create figure and subplot with Aitoff projection
    plt.figure(figsize=figsize)
    ax = plt.subplot(111, projection='aitoff')
    
    # Convert coordinates to SkyCoord object
    coords = SkyCoord(ra=df[ra_col].values*u.degree, 
                     dec=df[dec_col].values*u.degree)
    
    # Convert to radians and flip RA
    ra_rad = -coords.ra.wrap_at(180*u.deg).radian
    dec_rad = coords.dec.radian
    
    # Set up scatter plot parameters
    scatter_kwargs = {'alpha': alpha}
    
    # Add colors if specified
    if color_col is not None:
        scatter_kwargs['c'] = df[color_col]
        
    # Add sizes if specified
    if size_col is not None:
        scatter_kwargs['s'] = df[size_col]
    
    # Create scatter plot
    scatter = ax.scatter(ra_rad, dec_rad, **scatter_kwargs)
    
    # Add colorbar if using colors
    if color_col is not None:
        plt.colorbar(scatter, label=color_col)
    
    # Set up grid and labels
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Right Ascension')
    ax.set_ylabel('Declination')
    
    # Set RA labels
    ra_ticks = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    ra_labels = ['150°', '120°', '90°', '60°', '30°', '0°', 
                '330°', '300°', '270°', '240°', '210°']
    ax.set_xticklabels(ra_labels)
    
    # Add declination labels
    dec_ticks = np.array([-60, -30, 0, 30, 60])
    dec_labels = [f'{tick}°' for tick in dec_ticks]
    ax.set_yticks(np.radians(dec_ticks))
    ax.set_yticklabels(dec_labels)
    
    plt.title(title)
    
    return ax

# Example usage:
"""
# Basic usage:
create_sky_plot(df)

# With color coding:
create_sky_plot(df, color_col='magnitude')

# With both color and size coding:
create_sky_plot(df, color_col='magnitude', size_col='mass')

# Full example with custom parameters:
fig = plt.figure(figsize=(15, 10))
ax = create_sky_plot(df,
                    ra_col='RA',
                    dec_col='DEC',
                    color_col='magnitude',
                    size_col='mass',
                    alpha=0.6,
                    title='Sky Distribution of Objects')

# Add additional features
plt.grid(True, alpha=0.3, linestyle='--')

# Customize colorbar
if 'magnitude' in df.columns:
    plt.clim(df['magnitude'].min(), df['magnitude'].max())
    plt.colorbar(label='Magnitude')

plt.show()
"""

# For density plots:
def create_density_sky_plot(df, ra_col='RA', dec_col='DEC', 
                          bins=50, figsize=(12, 8)):
    """
    Create a density plot using hexbins
    """
    plt.figure(figsize=figsize)
    ax = plt.subplot(111, projection='aitoff')
    
    coords = SkyCoord(ra=df[ra_col].values*u.degree, 
                     dec=df[dec_col].values*u.degree)
    
    ra_rad = -coords.ra.wrap_at(180*u.deg).radian
    dec_rad = coords.dec.radian
    
    # Create hexbin plot
    hb = ax.hexbin(ra_rad, dec_rad, gridsize=bins, 
                   cmap='viridis', bins='log')
    
    plt.colorbar(hb, label='log10(N)')
    
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Right Ascension')
    ax.set_ylabel('Declination')
    
    # Set RA labels
    ra_ticks = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    ra_labels = ['150°', '120°', '90°', '60°', '30°', '0°', 
                '330°', '300°', '270°', '240°', '210°']
    ax.set_xticklabels(ra_labels)
    
    plt.title('Sky Density Distribution')
    plt.show()

In [ ]:
create_sky_plot(df, ra_col='fieldRA', dec_col='fieldDec', 
                   color_col=None, size_col=None, 
                   figsize=(12, 8), alpha=0.5,
                   title='All-Sky Aitoff Projection')

In [ ]:
create_density_sky_plot(df, ra_col='fieldRA', dec_col='fieldDec', 
                          bins=200, figsize=(12, 8))

In [ ]:
%pwd

In [ ]:
df.to_csv('LSSTcam_pointings.baseline_v4.0_10yrs.csv')